#### Testing and Implementation Guidelines for Project 4
CS 236 <br>
Fall 2024

Mike Goodrich and Eric Mercer <br>
Brigham Young University <br>
March 2023 and November 2024

---

Project 4 has historically been the most challenging project. The purpose of this Jupyter notebook is to help in two ways:
- demonstrate a connection the high-level discussion of natural join and an implementation of a specific case
- discuss testing strategies for `Interpreter.union` and `Interpreter.join`
- give examples of what happens when the *fixed point algorithm* runs. This algorithm is described in the Project 4 specs
- give some example test for evaluating the fixed point algoirthm

---

#### Testing Union

Let's begin with a discussion of testing the `union` operator. The code in the next two cells is taken from the Project 3 Jupyter notebook.

Note that I'm running this Jupyter notebook code inside a virtual environment in which pytest and tabulate have been installed.

In [1]:
class IncompatibleOperandError(Exception):
    """ Class for handling exceptions caused by mismatched relation types
    """
    def __init__(self, msg: str) -> None:
        super().__init__(msg)

The following implementation of the `union` operator does not do everything needed for project 4. 

In [2]:
from typing import Any
from tabulate import tabulate

RelationTuple = tuple[str | int, ...]

class Relation:
    def __init__(self, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )
    
    ########################
    # Relational Operators #
    ########################
    def union(self,other) -> 'Relation':
        if not isinstance(other, Relation):
            raise IncompatibleOperandError(f"Tried to union a relation with a {type(other)}") # don't attempt to union with something not a relation
        # First, check the precondition to see if the headers are the same
        if self.header != other.header:
            raise IncompatibleOperandError("Tried to union two relations with different headers")
        
        # Second, create a new header that is the union of the sets of tuples
        header: list[str] = self.header
        set_of_tuples = self.set_of_tuples
        set_of_tuples = set_of_tuples.union(other.set_of_tuples)    # This is the union operator defined for set objects
        return Relation(header,set_of_tuples) # Create a new relation


Let's talk about writing tests for the union function. We should write at least one *positive* test and at least one *negative* test. Below is the copilot response to the query "What's the difference between a positive unit test and a negative unit test?"

---

In software testing, **unit tests** are used to verify that individual components of the software work as expected. Here's the difference between **positive** and **negative** unit tests:

1. **Positive Unit Tests**:
   - These tests check if the software behaves correctly with valid input.
   - They ensure that the component produces the expected result when given valid data.
   - For example, if you have a function that adds two numbers, a positive test would check if `add(2, 3)` returns `5`.

2. **Negative Unit Tests**:
   - These tests check how the software handles invalid or unexpected input.
   - They ensure that the component fails gracefully or handles errors properly.
   - For example, using the same `add` function, a negative test might check if `add(null, 3)` throws an appropriate error or returns a specific error code.

Both types of tests are crucial for ensuring the robustness and reliability of your software. Do you have a specific scenario in mind where you're thinking of applying these tests?

---

One **negative test** for the `union` function is one that evaluates what happens when we try to union a relation with something that is not a relation. Another **negative test** is one that evaluates what happens when we try to union to relations that have different headers. Let's write a test for each.

Since I'm fairly new to writing unit tests, I'm going to ask for help from copilot. Here's the prompt I used: 

*I'd like help writing unit tests for the union function in the following Relation class in python using pytest*. 

I then I modified the test returned by copilot in four ways, two ways that make the interaction in the Jupyter notebook work better, and two ways that cause the tet to match the pattern given in the project 3 tests `tests/test_relation.py`:
- I added the message "Incorrect error message for mismatched headers in Relation.union" when the test fails. Since I'm running the tests interactively, I wanted something informative printed out when things fail.
- I added the print statement after the assert statement, which runs if the assert statement succeeds. This print statement tells me that the test ran successfully. I've done this since I'm not running my tests inside a vscode test environment. 
- I added comments that match the comments in the starter code tests
- I "caught" the object processed by `python.raises` and named it `exception`


In [3]:
import pytest

def test_union_different_headers():
    # What I expect
    expected_error_message: str = "Tried to union two relations with different headers"

    # Instantiate two relations
    relation1 = Relation(['id', 'name'], {('1', 'Alice'), ('2', 'Bob')})
    relation2 = Relation(['id', 'age'], {('3', 'Charlie'), ('4', 'David')})
    
    # when
    with pytest.raises(IncompatibleOperandError) as exception:
        _ = relation1.union(relation2)

    # then
    assert (
        expected_error_message == str(exception.value)
    ) , f"Incorrect error message \"{exception.value}\" for mismatched headers in Relation.union"
    
    # If assertion satisfied
    print("test_union_different_header succeeded")

test_union_different_headers()


test_union_different_header succeeded


Try to change the `expected_error_message` to something else, rerun the test, and think about what happened.

---

Notice the pattern in this negative test:
- define a test function
- tell the test function what error message to expect for this negative test
- instantiate two relations
- capture the `exception` that is `raise`d when the `union` function is called
- assert that the expected error message matches the error message returned from `union`
- run the test by calling `test_union_different_headers()`

Line 12 is the code that captures the exception. It might help to look at the things that make up that line:
- The keyword `with` tells us that we are handling irregular program termination in our negative test
- The code segment `pytest.raises` indicates that we are looking for a particular exception being raised
- The argument `IncompatibleOperandError` is the kind of exception we are looking for
- The code segment `as exception` says that we want to capture the object created by the exception in a variable called `exception`

The underscore in Line 13 says that we know the union operator returns something, but we'll never use it so we aren't going to name it.

Line 17 uses the `value` stored in the `exception` object. This `value` is the error message that is written when the exception occurs. Line 17 compares the actual error message with the error message that we knew should occur during the negative test.

---

The second test returned by copilot evaluates what happens when I try to union a relation with something that is not a relation. Again, I modified the test to better illustrate things and to match the pattern given in the project 3 tests `tests/test_relation.py`.

In [4]:
def test_union_invalid_type():
    # What I expect
    expected_error_message: str = "Tried to union a relation with a <class 'str'>"
    
    # Instantiate two objects
    relation1 = Relation(['id', 'name'], {('1', 'Alice'), ('2', 'Bob')})
    not_a_relation: str = "This is a string not a relation"
    
    # when
    with pytest.raises(IncompatibleOperandError) as exception:
        _ = relation1.union(not_a_relation)

    # then
    assert (
        expected_error_message == str(exception.value)
    ) , f"Unexpected error message in Relation.union {exception.value}"
    
    # If assertion satisfied
    print("test_union_invalid_type succeeded")

test_union_invalid_type()


test_union_invalid_type succeeded


Notice that I had to know a bit about what error would be printed to get this to work, but since we defined the error message in the class it's not too hard to get the expected error message correctly. Try modifying the error message in the `Relation` class and rerun the test. What happens?

---

Let's now do a **Positive Test**. This is where the code meets the math. Recall that two relations can only be "unioned" if the headers are the same. When the headers are the same, we say that the two relations are *union compatible*. 

Let's define two *union compatible* relations and do the math to create the union. We'll then see if the code generates the same relation as the math.

Consider the relation R 
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 

and consider a new relation Q defined as
| char | int | 
| :-: | :-: | 
| $f$ | $3$ |

The union $P\cup Q$ is 
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $f$ | $3$ | 

Let's test. We'll do this creating a relation that is formed from the math (i.e., the `expected_answer`), creating a P relation, and creating a Q relation. We'll then run the code and see if the relation returned (i.e.,  `given_answer`) matches the `expected_answer`.

In [5]:
import pytest

def test_union_nonempty_relations():
    # expected answer
    expected_answer: Relation = Relation(['char', 'name'], {('a','1'), ('b','2'), ('c','3'), ('f','3')})

    # Instantiate the two relations from the math
    R: Relation = Relation(['char', 'name'], {('a','1'), ('b','2'), ('c','3')})
    Q: Relation = Relation(['char', 'name'], {('f','3')})
    
    # when
    given_answer: Relation = R.union(Q)

    # then
    assert (
        expected_answer == given_answer
    ) , f"Expected answer = {expected_answer}\nGiven answer = {given_answer}"
    
    # If assertion satisfied
    print("Successfully implemented union for this test")

test_union_nonempty_relations()

Successfully implemented union for this test


We can now try two write tests that could arise in the integration tests, like empty relations. I'll skip showing the math except to say that the union of two empty relations that are union compatible is also an empty relation.

In [6]:
import pytest

def test_union_empty_relations():
    # expected answer
    expected_answer: Relation = Relation(['char', 'name'], set())

    # Instantiate the two relations from the math
    R: Relation = Relation(['char', 'name'], set())
    Q: Relation = Relation(['char', 'name'], set())
    
    # when
    given_answer: Relation = R.union(Q)

    # then
    assert (
        expected_answer == given_answer
    ) , f"Expected answer = {expected_answer}\nGiven answer = {given_answer}"
    
    # If assertion satisfied
    print("Successfully implemented union for this test")

test_union_empty_relations()

Successfully implemented union for this test


---

#### Implementing and Testing Join

The pseudo-code below is a modified version of the pseudo-code shown in class and given in the Project 4 specifications. The pseudo-code is only describes one form of `join`, namely the `join` when the headers of the two relations are identical.


<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
1 create header <font color = "lightblue">my_header</font><br>
2 create empty relation <font color = "lightblue">new_relation</font> with header <font color = "lightblue">my_header</font><br>
3 for each tuple <font color = "lightblue">tuple_1</font> in <font color = "lightblue">relation_1</font><br>
4 &nbsp;&nbsp;&nbsp;&nbsp; for each tuple <font color = "lightblue">tuple_2</font> in <font color = "lightblue">relation_2</font><br>
5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if <font color = "lightblue">tuple_1</font> and <font color = "lightblue">tuple_2</font> can join <br>
6 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<font color = "lightblue">new_tuple</font> = <font color = "red">join</font>(<font color = "lightblue">tuple_1</font>,<font color = "lightblue">tuple_2</font>)<br>
7 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
add <font color = "lightblue">new_tuple</font> to <font color = "lightblue">new_relation</font>
</div>


I've implemented a join operator for this special case below. The code includes the `add_tuple` function from the Project 3 starter code, and the code does not include `union` to keep it small enough to read easily. 

In [7]:
from typing import Any
from tabulate import tabulate

RelationTuple = tuple[str | int, ...]

class Relation:
    def __init__(self, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )
    
    def add_tuple(self, r: RelationTuple) -> None:
        if len(self.header) != len(r):
            raise IncompatibleOperandError(
                f"Error: {r} is not compatible with header {self.header} in Relation.add_tuple"
            )
        if not isinstance(r, tuple) or any(not isinstance(i, str) for i in r):
            raise IncompatibleOperandError(
                f"Error: {r} is not type compatible with Relation.RelationTuple in Relation.add_tuple"
            )
        self.set_of_tuples.add(r)
    
    ########################
    # Relational Operators #
    ########################
    def join_identical_headers(self,other) -> 'Relation':
        # Check to see if joining with a relation and if the headers are identical
        if not isinstance(other, Relation):
            raise IncompatibleOperandError(f"Tried to union a relation with a {type(other)}") # don't attempt to union with something not a relation
        if self.header != other.header:
            raise IncompatibleOperandError("Tried to union two relations with different headers")
        
        # Pseudo-code line 1: create the header for the new relation
        new_header: list[str] = self.header
        # Pseudo-code line 2: create empty relation
        new_relation: Relation = Relation(relation_header=new_header, set_of_tuples= set())
        # Pseudo-code lines 3 and 4: iterate through each tuple in each relation
        for tuple_1 in self.set_of_tuples:
            for tuple_2 in other.set_of_tuples:
                # Pseudo-code line 6 and 7. 
                # Recall that if the headers are identical then natural join becomes intersection
                # Thus, only add a tuple if it is in both relations
                if tuple_1 == tuple_2:
                    new_relation.add_tuple(tuple_1)
        
        return new_relation


You should be able to write **negative tests** for `Relation.join` using the pattern shown above for `Relation.union`.

Let's write a **positive est** for `Relation.join`. We begin by "doing the math."

Natural join is equivalent to intersection if the attributes of the two relations are identical. 

Consider the relation R1 defined as 
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ | 

and consider a new relation R2 defined as
| char | int | 
| :-: | :-: | 
| $f$ | $3$ |
| $b$ | $2$ | 
| $b$ | $3$ | 
| $d$ | $4$ | 

According to class lecture, when the headers of the two relations match the output $R1 \bowtie R2$ is equivalent to $R1 \cap R2$, which yields
| char | int | 
| :-: | :-: | 
| $b$ | $2$ | 
| $d$ | $4$ | 


We can now write the test because we know the expected answer.

In [8]:
import pytest

def test_join_identical_headers():
    # expected answer
    expected_answer: Relation = Relation(['char', 'int'], {('b','2'), ('d','4')})

    # Instantiate the two relations from the math
    R1: Relation = Relation(['char', 'int'], {('a','1'), ('b','2'), ('c','3'), ('d','4')})
    R2: Relation = Relation(['char', 'int'], {('f','3'), ('b','2'), ('b','3'), ('d','4')})
    
    # when
    given_answer: Relation = R1.join_identical_headers(R2)
    print(given_answer)

    # then
    assert (
        expected_answer == given_answer
    ) , f"Expected answer = {expected_answer}\nGiven answer = {given_answer}"
    
    # If assertion satisfied
    print("Successfully implemented join for this test")

test_join_identical_headers()

╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ b      │     2 │
├────────┼───────┤
│ d      │     4 │
╘════════╧═══════╛
Successfully implemented join for this test


The pattern in the test is:
- Instantiate the relation that is the answer to the math we did and name it `expected_relation`
- Instantiate the other two relations used in the math, `R1` and `R2`
- Execute the `join` function and name the relation created by the `join` in `given_answer`
- Compare the `expected_answer` to the `given_answer`

---

Let's write a different test, this time for when there are no common tuples in the two relations.

Consider the relation R3 defined as 
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 

and consider a new relation R4 defined as
| char | int | 
| :-: | :-: | 
| $f$ | $3$ |
| $b$ | $3$ | 
| $d$ | $4$ | 

According to class lecture, when the headers of the two relations match the output $R3 \bowtie R4$ is equivalent to $R3 \cap R4$, which yields
| char | int | 
| :-: | :-: | 



In [9]:
def test_join_identical_headers_empty_relation():
    # expected answer
    expected_answer: Relation = Relation(['char', 'int'], set())

    # Instantiate the two relations from the math
    R3: Relation = Relation(['char', 'int'], {('a','1'), ('b','2')})
    R4: Relation = Relation(['char', 'int'], {('f','3'), ('b','3')})
    
    # when
    given_answer: Relation = R3.join_identical_headers(R4)
    print(given_answer)

    # then
    assert (
        expected_answer == given_answer
    ) , f"Expected answer = {expected_answer}\nGiven answer = {given_answer}"
    
    # If assertion satisfied
    print("Successfully implemented join for this test")

test_join_identical_headers_empty_relation()

╒════════╤═══════╕
│ char   │ int   │
╞════════╪═══════╡
╘════════╧═══════╛
Successfully implemented join for this test


The pattern in the test is the same as the previous test:
- Instantiate the relation that is the answer to the math we did and name it `expected_relation`
- Instantiate the other two relations used in the math, `R1` and `R2`
- Execute the `join` function and name the relation created by the `join` in `given_answer`
- Compare the `expected_answer` to the `given_answer`

---

#### Parameterized Tests

Let's now explore writing *parameterized tests*. The idea of a parameterized test is that we want to run the same test but with a different set of input conditions. The different input conditions are the *parameters* of the test.

In pytest, we can write a parameterized test by *decorating the test*. You don't have to know exactly what a *decorator* is as long as you know the pattern. (If you want to learn about *decorators*, consider the tutorial at https://realpython.com/primer-on-python-decorators/ .) We'll rewrite the two tests above into a single function that takes three arguments: the first relation, the second relation, and the relation produced by doing the math. 

In [10]:
def test_join_identical_headers(R1: Relation, R2: Relation, expected_answer: Relation) -> None:
    # when
    given_answer: Relation = R1.join_identical_headers(R2)
    print(given_answer)

    # then
    assert (
        expected_answer == given_answer
    ) , f"Expected answer = {expected_answer}\nGiven answer = {given_answer}"
    
    # If assertion satisfied
    print("Successfully implemented join for this test")


Let's follow the pattern in the `tests/test_datalogprogram.py` file in Project 2. We'll add a decorator to `test_join_identical_headers` by adding a `pytest.mark.parametrize`. This decorator has three arguments: `argnames`, `argvalues`, and `ids`. The argument names will contain the list of arguments that will be passed to `test_join_identical_headers` yielding

    argnames = ["R1", "R2", "expected_answer"]

The argument values will be instances of the different relations that will be passed in, a different set of relations for each test. We have two sets: the triple $(R1, R2, R1 \bowtie R2)$ and $(R3, R4, R3 \bowtie R4)$, which match the relations we used "in the math", yielding

    (Relation(['char', 'int'], {('a','1'), ('b','2'), ('c','3'), ('d','4')}),
    Relation(['char', 'int'], {('f','3'), ('b','2'), ('b','3'), ('d','4')}),
    Relation(['char', 'int'], {('b','2'), ('d','4')})
    )

and

    (Relation(['char', 'int'], {('a','1'), ('b','2')}),
    Relation(['char', 'int'], {('f','3'), ('b','3')}),
    Relation(['char', 'int'], set())
    )

The ids will be informative names that tell us somethign about the test. The first test had a nonempty relation as the answer, and the second test had an empty relation as the answer yielding

    ids = ["nonempty", "empty"]

Let's put the pieces together.



In [ ]:
# Define things to be passed in as argument values
R1_join_R2: Relation = Relation(['char', 'int'], {('b','2'), ('d','4')})
R1: Relation = Relation(['char', 'int'], {('a','1'), ('b','2'), ('c','3'), ('d','4')})
R2: Relation = Relation(['char', 'int'], {('f','3'), ('b','2'), ('b','3'), ('d','4')})

R3_join_R4: Relation = Relation(['char', 'int'], set())
R3: Relation = Relation(['char', 'int'], {('a','1'), ('b','2')})
R4: Relation = Relation(['char', 'int'], {('f','3'), ('b','3')})

@pytest.mark.parametrize(
       argnames = ["R1", "R2", "expected_answer"],
       argvalues = [
           (R1, R2, R1_join_R2),
           (R3, R4, R3_join_R4)
       ],
       ids = ["nonempty", "empty"] 
)
def test_join(R1: Relation, R2: Relation, expected_answer: Relation) -> None:
    # when
    given_answer: Relation = R1.join_identical_headers(R2)
    print(given_answer)

    # then
    assert (
        expected_answer == given_answer
    ) , f"Expected answer = {expected_answer}\nGiven answer = {given_answer}"
    
    # If assertion satisfied
    #print("Successfully implemented join for this test")


I've run these parameterized tests using the methods from the Projects and they work. I've commented out the information used to print things from the previous examples. It's a pain to run parameterized tests in a Jupyter notebook and talking about how to do them would probably do more harm than good. 

Please note the misspelling of *parameterize* in `@pytest.mark.parametrize`. The misspelled version is what is required for pytest so don't do what I did and spell it correctly only to spend time trying to figure out why the test wasn't running.

---

Let's recap:
- Negative tests are designed to make sure your code fails the right way when something goes wrong. The key code element is the segment `with python.raises(error_type)`.
- Positive tests are designed to make sure the implementation of whatever unit you are creating in your code actually does what you think it should. The best practice is to write the tests before you implement the unit. That does two things: it helps solidify what you want the unit to do before you start coding, and it allows incremental testing of functionality. The key code element in the test is the `assert` statement
- A parameterized test acknowledges the fact that there can be a lot of different things that need to be done depending on what is passed into the unit. The parameterized test runs a single test but with a lot of different inputs. The key code element is the decorator `@pytest.mark.parametrize()` that specifies the different inputs to be tested along with what the different inputs should produce

---

#### Some Ideas for Testing Join

Let's create a list of the kinds of things that we want natural join to handle.
- We want it to act like intersection if all the headers of the two relations match
- We want it to act like cross product if all the headers of the two relations are different
- We want it to merge headers and tuples if the headers of the two relations have a common attribute
- We want it to merge headers and tuples if the headers of the two relations have more than one common attribute
- We want it to merge headers if the attributes of the two headers overlap but are in different orders

From this list, we can start to create some tests:



**Matching Headers** 

See the parameterized tests above. 

Also consider a test like create the new relation $R' = R\bowtie Q$ when 

the relation $R$ is
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 

and the relation $Q$ is
| int | char |
| :-: | :-: | 
| $1$ | $a$ | 
| $3$ | $f$ |

Notice how the order of the attributes is different but the attributes are the same. 

You'd create the test by doing the math to find $R'$. *Hint: the header for the relation $R\bowtie Q$ is `['char', 'int']`.*

---

**Non-overlapping Attributes** 

Create the new relation $R' = R\bowtie Q$ when

the relation $R$ is
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 

and the relation $Q$ is
| A   | B   |
| :-: | :-: | 
| $1$ | $a$ | 
| $3$ | $f$ |

Notice how no attributes in the headers are the same between the two relations.

You'd create the test by doing the math to find $R'$.

---

**One Overlapping Attribute** 

Create the new relation $R' = R\bowtie Q$ when 

the relation $R$ is
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 

and the relation $Q$ is
| int   | B   |
| :-: | :-: | 
| $1$ | $a$ | 
| $3$ | $f$ |

Notice how the *int* attribute in each header is the same.

---

**Multiple Overlapping Attribute** 

Create the new relation $R' = R\bowtie Q$ when 

the relation $R$ is
| A | B | C |
| :-: | :-: | :-:| 
| $a$ | $1$ | $x$ |
| $b$ | $2$ | $y$ |

and the relation $Q$ is
| A   | B   | D |
| :-: | :-: | :-: |
| $a$ | $1$ | $q$ |
| $b$ | $3$ | $r$ |

Notice how both headers have an *A* attribute and a *B* attribute.

---

**Multiple Out-of-Order Overlapping Attribute** 

Create the new relation $R' = R\bowtie Q$ when 

the relation $R$ is
| A | B | C |
| :-: | :-: | :-:| 
| $a$ | $1$ | $x$ |
| $b$ | $2$ | $y$ |

and the relation $Q$ is
| D   | B   | A |
| :-: | :-: | :-: |
| $q$ | $1$ | $a$ |
| $r$ | $3$ | $b$ |

Notice how both headers have an *A* attribute and a *B* attribute but the order is different.

---
---

#### The Fixed-Point Algorithm

Let's now talk about the fixed point algorithm. The basic idea is that rules use tuples from the database to create new tuples to add to the database. This means that you might need to evaluate the rule multiple times. Let's illustrate with an example. Consider the following datalog program which uses some names from my family history

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
Schemes:<br>
&nbsp;&nbsp;&nbsp;&nbsp; marriedTo(Person, Partner)<br>
<br>
Facts: <br>
&nbsp;&nbsp;&nbsp;&nbsp; marriedTo('Mae', 'Wilder').<br>
&nbsp;&nbsp;&nbsp;&nbsp; marriedTo('Levi', 'Evelyn').<br>
<br>
Rules: <br>
&nbsp;&nbsp;&nbsp;&nbsp; marriedTo(A, B)&nbsp; :- &nbsp; marriedTo(B,A). <br>
<br>
Queries: <br>
&nbsp;&nbsp;&nbsp;&nbsp; marriedTo(Spouse1, Spouse2)? <br>
</div>



Evaluating schemes and facts creates the *marriedTo* relation

| Person   | Partner   |
| :-: | :-: | 
| 'Mae' | 'Wilder' | 
| 'Levi' | 'Evelyn' |

Evaluating the rule once adds two tuples to the *marriedTo* relation

| Person   | Partner   |
| :-: | :-: | 
| 'Mae' | 'Wilder' | 
| 'Levi' | 'Evelyn' |
| 'Wilder' | 'Mae' | 
| 'Evelyn' | 'Levi' |

which makes sense because the rule says that if person $A$ is married to person $B$ then person $B$ is married to person $A$.

We know that this is what the rule says because we can interpret the content of the datalog program. Because we know this, we also know that we only need to evaluate the rules once.  But the `Interpreter` does not know this. Consequently, the `evaluate_rules` function does the natural join and union again to see if any new tuples are added the second time. This yields the same relation

*marriedTo* relation

| Person   | Partner   |
| :-: | :-: | 
| 'Mae' | 'Wilder' | 
| 'Levi' | 'Evelyn' |
| 'Wilder' | 'Mae' | 
| 'Evelyn' | 'Levi' |

Since no tuples have been added to the relation, the algorithm has reached a *fixed point* and knows to terminate.

Evaluating the query produces the relation

| Spouse1  | Spouse2  |
| :-: | :-: | 
| 'Mae' | 'Wilder' | 
| 'Levi' | 'Evelyn' |
| 'Wilder' | 'Mae' | 
| 'Evelyn' | 'Levi' |

---

Let's look at a second datalog program

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
Schemes:<br>
&nbsp;&nbsp;&nbsp;&nbsp; olderThan(Sibling1, Sibling2)<br>
<br>
Facts: <br>
&nbsp;&nbsp;&nbsp;&nbsp; olderThan('Levi', 'Anna').<br>
&nbsp;&nbsp;&nbsp;&nbsp; olderThan('Anna', 'McKay').<br>
&nbsp;&nbsp;&nbsp;&nbsp; olderThan('McKay', 'June').<br>
&nbsp;&nbsp;&nbsp;&nbsp; olderThan('June', 'Gene').<br>
<br>
Rules: <br>
&nbsp;&nbsp;&nbsp;&nbsp; olderThan(A, C)&nbsp; :- &nbsp; olderThan(A,B) &nbsp; , &nbsp; olderThan(B,C).
<br>
Queries: <br>
&nbsp;&nbsp;&nbsp;&nbsp; olderThan(Sibling, 'Gene')? <br>
</div>

Evaluating schemes and facts produces the relation

| Sibling1   | Sibling2   |
| :-: | :-: | 
| 'Levi' | 'Anna' | 
| 'Anna' | 'McKay' |
| 'McKay' | 'June' |
| 'June' | 'Gene' |

Evaluating the rule one time yields

| Sibling1   | Sibling2   |
| :-: | :-: | 
| 'Levi' | 'Anna' | 
| 'Levi' | 'McKay' |
| 'Anna' | 'McKay' |
| 'Anna' | 'June' |
| 'McKay' | 'June' |
| 'McKay' | 'Gene' |
| 'June' | 'Gene' |

I did some sorting on the tuples to help me visually track what is happening.

Evaluating the rule a second time yields

| Sibling1   | Sibling2   |
| :-: | :-: | 
| 'Levi' | 'Anna' | 
| 'Levi' | 'McKay' |
| 'Levi' | 'June' |
| 'Anna' | 'McKay' |
| 'Anna' | 'June' |
| 'Anna' | 'Gene' |
| 'McKay' | 'June' |
| 'McKay' | 'Gene' |
| 'June' | 'Gene' |

Evaluating the rule the third time yields

| Sibling1   | Sibling2   |
| :-: | :-: | 
| 'Levi' | 'Anna' | 
| 'Levi' | 'McKay' |
| 'Levi' | 'June' |
| 'Levi' | 'Gene' |
| 'Anna' | 'McKay' |
| 'Anna' | 'June' |
| 'Anna' | 'Gene' |
| 'McKay' | 'June' |
| 'McKay' | 'Gene' |
| 'June' | 'Gene' |

Evaluating the rule the fourth time yields

| Sibling1   | Sibling2   |
| :-: | :-: | 
| 'Levi' | 'Anna' | 
| 'Levi' | 'McKay' |
| 'Levi' | 'June' |
| 'Levi' | 'Gene' |
| 'Anna' | 'McKay' |
| 'Anna' | 'June' |
| 'Anna' | 'Gene' |
| 'McKay' | 'June' |
| 'McKay' | 'Gene' |
| 'June' | 'Gene' |

which has no changes. We've reached the *fixed point* and can end.

Evaluating the query yields the relation

| Sibling   |
| :-: | 
| 'Levi' | 
| 'Anna' | 
| 'McKay' |
| 'June' | 

---

These examples suggest some test ideas.

**Test on the marriedTo Relation**

- Check to see if the rules are evaluated twice
- When the fixed point algorithm terminates and before evaluating queries, check to see if the marriedTo relation is

| Person   | Partner   |
| :-: | :-: | 
| 'Mae' | 'Wilder' | 
| 'Levi' | 'Evelyn' |
| 'Wilder' | 'Mae' | 
| 'Evelyn' | 'Levi' |

**Test on the olderThan Relation**

- Check to see if the rules are evaluated four times
- When the fixed point algorithm terminates and before evaluating queries, check to see if the olderThan relation is

 Sibling1   | Sibling2   |
| :-: | :-: | 
| 'Levi' | 'Anna' | 
| 'Levi' | 'McKay' |
| 'Levi' | 'June' |
| 'Levi' | 'Gene' |
| 'Anna' | 'McKay' |
| 'Anna' | 'June' |
| 'Anna' | 'Gene' |
| 'McKay' | 'June' |
| 'McKay' | 'Gene' |
| 'June' | 'Gene' |